# User Days 2020 NWB Tutorial - Optical Physiology

## Introduction
In this tutorial, we will create fake data for a hypothetical optical physiology experiment with a freely moving animal. The types of data we will convert are:
* Subject (species, strain, age, etc.)
* Animal position
* Trials
* Acquired two-photon images
* Image segmentation (ROIs)
* Fluorescence and dF/F response

## Installing PyNWB
If you are in the tutorial using DANDI Hub, PyNWB is already installed. 
If participating from your own machine, install PyNWB using pip or conda:
- `pip install pynwb`
- `conda install -c conda-forge pynwb`

## Set up the NWB file
An NWB file represents a single session of an experiment. Each file must have a session description, identifier, and session start time. Create a new `NWBFile` object with those and additional metadata. For all PyNWB constructors, we recommend using keyword arguments.

In [ ]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz('US/Pacific'))

nwb = NWBFile(session_description='Head-fixed mouse on a wheel',
              identifier='Mouse5_Day3',
              session_start_time=start_time,
              session_id='session_1234',                                # optional
              experimenter='My Name',                                   # optional
              lab='My Lab Name',                                        # optional
              institution='University of My Institution',               # optional
              related_publications='DOI:10.1016/j.neuron.2016.12.011')  # optional
nwb

## Subject information

Create a `Subject` object to store information about the experimental subject, such as age, species, genotype, sex, and a freeform description. And set `nwb.subject` to the `Subject` object.

In [ ]:
from pynwb.file import Subject

nwb.subject = Subject(subject_id='0001',
                      age='P9M', 
                      description='mouse 5',
                      species='Mus musculus', 
                      sex='M')

## SpatialSeries
`SpatialSeries` is a subclass of `TimeSeries`. `TimeSeries` is a common base class for measurements sampled over time, and provides fields for data and time (regularly or irregularly sampled). The `Position` object is a special type of object called a `MultiContainerInterface`. It holds one or more `SpatialSeries` objects. 

<img src="../images/spatialseries.svg">

Create a `SpatialSeries` object named `'position'` with some fake data.

In [ ]:
import numpy as np
from pynwb.behavior import SpatialSeries, Position

position_data = np.array([np.linspace(0, 10, 100),
                          np.linspace(1, 8, 100)]).T
spatial_series_object = SpatialSeries(
    name='position', 
    data=position_data,
    reference_frame='unknown',
    timestamps=np.linspace(0, 100) / 200)

Then, create a `Position` object which contains the `SpatialSeries` object you just created.

In [ ]:
pos_obj = Position(spatial_series=spatial_series_object)

Finally, create a processing module named `'behavior'` in the NWB file and add the `Position` object to the processing module.

In [ ]:
behavior_module = nwb.create_processing_module(
    name='behavior',
    description='processed behavioral data')

behavior_module.add(pos_obj)

## Write to file

In [ ]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('ecephys_tutorial.nwb', 'w') as io:
    io.write(nwb)

## Trials

<img src="../images/trials.svg">

`DynamicTable` objects are used to store tabular metadata throughout NWB, including electrodes and sorted units. They offer flexibility for tabular data by allowing required columns, optional columns, and custom columns.

Trials are stored in a `TimeIntervals` object which subclasses `DynamicTable`. Here, we are adding a column named `'correct'`, which will be a boolean array.

In [ ]:
nwb.add_trial_column(name='correct', description='whether the trial was correct')
nwb.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwb.add_trial(start_time=6.0, stop_time=10.0, correct=False)

## Optical physiology
Optical physiology results are written in four steps:
1. Create imaging plane
2. Acquired two-photon images
3. Image segmentation
4. Fluorescence and dF/F responses

### Imaging Plane
First, you must create an `ImagingPlane` object, which will hold information about the area and method used to collect the optical imaging data. This requires creation of a `Device` object for the microscope and an `OpticalChannel` object. Then you can create an `ImagingPlane`.


<img src="../images/imagingplane.svg">


In [ ]:
from pynwb.device import Device
from pynwb.ophys import OpticalChannel

device = Device('Microscope001')
nwb.add_device(device)
optical_channel = OpticalChannel('OpticalChannel', 'my_description', 500.)
imaging_plane = nwb.create_imaging_plane(
    name='ImagingPlane',
    optical_channel=optical_channel,
    imaging_rate=30.,
    description='a very interesting part of the brain',
    device=device,
    excitation_lambda=600.,
    indicator='GFP',
    location='V1',
    grid_spacing=[.01, .01],
    grid_spacing_unit='meters')

### TwoPhotonSeries
Now that you have your `ImagingPlane`, you can create a `TwoPhotonSeries` the class representing two photon imaging data. From here you have two options. The first option is to supply the image data to PyNWB, using the `data` argument. The other option is the provide a path the images. These two options have trade-offs, so it is worth spending time considering how you want to store this data.

<img src="../images/twophotonseries.svg">


In [ ]:
from pynwb.ophys import TwoPhotonSeries

# using internal data. This data will be stored inside the HDF5 file
image_series1 = TwoPhotonSeries(name='TwoPhotonSeries1',
                               data=np.ones((1000,100,100)),
                               imaging_plane=imaging_plane,
                               rate=1.0)
nwb.add_acquisition(image_series1)

# using external data
image_series2 = TwoPhotonSeries(name='TwoPhotonSeries2', dimension=[2],
                               external_file=['images.tiff'], imaging_plane=imaging_plane,
                               starting_frame=[0], format='tiff', starting_time=0.0, rate=1.0)
nwb.add_acquisition(image_series2)

### Plane Segmentation
Image segmentation stores the detected regions of interest in the `TwoPhotonSeries` data. `ImageSegmentation` allows you to have more than one segmentation by creating more `PlaneSegmentation` objects.

<img src="../images/planesegmentation.svg" width="800">


In [ ]:
from pynwb.ophys import ImageSegmentation

mod = nwb.create_processing_module(
    name='ophys', description='contains optical physiology processed data')
img_seg = ImageSegmentation()
mod.add(img_seg)
ps = img_seg.create_plane_segmentation(
    name='PlaneSegmentation',  # required
    description='output from segmenting my favorite imaging plane',  # required
    imaging_plane=imaging_plane,  # required
    reference_images=image_series1)  # optional

### Regions Of Interest (ROIs)
#### image masks
You may add ROIs using an image mask or using a pixel mask. An image mask is an array that is the same size as a single frame of the `TwoPhotonSeries`, and indicates where a single region of interest is. This image mask may be boolean or continuous between 0 and 1.

In [ ]:
for _ in range(30):
    image_mask = np.zeros((100, 100))
    
    # randomly generate example image masks
    x = np.random.randint(0, 95)
    y = np.random.randint(0, 95)
    image_mask[x:x+5, y:y+5] = 1
    
    # add image mask to plane segmentation
    ps.add_roi(image_mask=image_mask)
    
# show one of the image masks
import matplotlib.pyplot as plt
plt.imshow(image_mask)

#### pixel masks
Alternatively, you could define ROIs using pixel mask, which defines each pixel individually with (x, y, val). All undefined pixels are assumed to be 0. But you need to decide and keep it consistent. You cannot add some by image_mask and some by pixel_mask.

In [ ]:
ps2 = img_seg.create_plane_segmentation(
    name='PlaneSegmentation2',  # required
    description='output from segmenting my favorite imaging plane',  # required
    imaging_plane=imaging_plane,  # required
    reference_images=image_series1)  # optional

for _ in range(30):
    
    # randomly generate example image masks
    x = np.random.randint(0, 95)
    y = np.random.randint(0, 95)
    
    pixel_mask = []
    for ix in range(x, x+5):
        for iy in range(y, y+5):
            pixel_mask.append((ix, iy, 1))
    
    # add pixel mask to plane segmentation
    ps2.add_roi(pixel_mask=pixel_mask)

## Storing fluorescence measurements

Now that ROIs are stored, you can store fluorescence dF/F data for these regions of interest. This type of data is stored using the `RoiResponseSeries` class. You will not need to instantiate this class directly to create objects of this type, but it is worth noting that this is the class you will work with after you read data back in.

<img src="../images/roiresponseseries.svg">


First, create a data interface to store this data in

In [ ]:
from pynwb.ophys import Fluorescence

fl = Fluorescence()
mod.add(fl)

Because this data stores information about specific ROIs, you will need to provide a reference to the ROIs that you will be storing data for. This is done using a `DynamicTableRegion`, which can be created with `PlaneSegmentation.create_roi_table_region`.

In [ ]:
rt_region = ps.create_roi_table_region(
    description='the first of two ROIs', region=[0,1])

Now that you have a `DynamicTableRegion`, you can create your an `RoiResponseSeries`.

In [ ]:
fl.create_roi_response_series(
    name='RoiResponseSeries',
    data=np.ones((100,2)),
    rois=rt_region,
    unit='lumens',
    rate=30.)